In [26]:
from pymongo import MongoClient
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from bson import ObjectId
import numpy as np

PRIVATE = "max:123"

user_id = "65d50ad01f7e4e1650e1baca"

def get_database():
 
   # Provide the mongodb atlas url to connect python to mongodb using pymongo
   CONNECTION_STRING = f"mongodb+srv://{PRIVATE}@test.ypmxjhw.mongodb.net"
  
   # Create a connection using MongoClient. You can import MongoClient or use pymongo.MongoClient
   client = MongoClient(CONNECTION_STRING)
   db = client.test
   listingColl = db.Listing
   userColl = db.User
   # Create the database for our example (we will use the same database throughout the tutorial
   return listingColl, userColl
  
#just do average simliariy of last 10% of listings and favorited



In [27]:

# This is added so that many files can reuse the function get_database()
if __name__ == "__main__":   
  
   # Get the database
    listingColl, userColl = get_database()

    listingCursor = listingColl.find()
    userCursor = userColl.find()
    
    #GETTING VECTORIZED FITTED ON THE WHOLE LISTINGS LIST
    allListingDescriptions = []
    for i in listingCursor:
        allListingDescriptions.append(i)
    allListingDescriptions = [listing['description'] for listing in allListingDescriptions]

    vectorizer = TfidfVectorizer()
    vectorizer.fit(allListingDescriptions + ["apprenticeship", "scholarships", "discounts", "funding", 'field trips', 'supplies', 'services', 'community'])

    #GETTING USER ACTIVITY 
    cursor = userColl.find({"_id": ObjectId("65d50ad01f7e4e1650e1baca")})
    favoriteIds = cursor[0]['favoriteIds']

    favoriteListings = []
    for i in favoriteIds:
        favoriteListings.append(listingColl.find({"_id": i})[0])

    latestListings = []
    cursorLatest = listingColl.find({"userId": ObjectId("65d50ad01f7e4e1650e1baca")}).sort("createdAt", -1)
    for i in cursorLatest[:1]:
        latestListings.append(i)

    listingDescriptions = [listing['description'] for listing in favoriteListings] 
    listingCategories = [listing['category'] for listing in favoriteListings] 

    combinedDescriptions = [f"{desc} {cat}" for desc, cat in zip(listingDescriptions, listingCategories)]
    
    listingDescriptions_tfidf = vectorizer.transform(combinedDescriptions)
    avg_tfidf = np.asarray(listingDescriptions_tfidf.mean(axis=0)).flatten()

    #COMPARISON
    explorableListings = listingColl.find({"userId": {"$ne": ObjectId("65d50ad01f7e4e1650e1baca")}})

    similarities = []

    for listing in explorableListings:
        description = f"{listing['description']} {listing['category']}"
        tfidf_vector = vectorizer.transform([description])
        similarity = cosine_similarity(tfidf_vector, avg_tfidf.reshape(1, -1))
        similarities.append((listing['_id'], similarity[0][0]))

    # Sort similarities in descending order
    sorted_similarities = sorted(similarities, key=lambda x: x[1], reverse=True)

    # Print or use sorted similarities as needed
    for listing_id, similarity in sorted_similarities:
        print(f"Listing ID: {listing_id}, Similarity with average: {similarity}")


Listing ID: 667e56eb0cf406d52b03b1c9, Similarity with average: 0.28729665600176785
Listing ID: 667e580b0cf406d52b03b1cc, Similarity with average: 0.15213259045724628
Listing ID: 667e56680cf406d52b03b1c7, Similarity with average: 0.12100741507864371
Listing ID: 667e579f0cf406d52b03b1cb, Similarity with average: 0.07754473645121186
Listing ID: 667e56b80cf406d52b03b1c8, Similarity with average: 0.07139835378353956
Listing ID: 667e576a0cf406d52b03b1ca, Similarity with average: 0.05108006511741294
Listing ID: 667e585a0cf406d52b03b1cd, Similarity with average: 0.04995224392606512
Listing ID: 667e58930cf406d52b03b1ce, Similarity with average: 0.04995224392606512
Listing ID: 667e591a0cf406d52b03b1d0, Similarity with average: 0.03823764472725008
Listing ID: 667e58dc0cf406d52b03b1cf, Similarity with average: 0.038188273234542186
